# Function to Tool Schema

> Python module to automatically convert a given Python function into tool schema appropriate for function calling

In [ ]:
#| default_exp core.fn_to_schema

In [ ]:
#| hide
from nbdev.showdoc import *

To execute function calling from GPT responses, we need to define the function and its schema as a GPT tool. However, it can be inconvenient to define both, especially if we need to incorporate multiple functions as GPT tools.

This process can be simplified if we utilize type-hinting and annotations on functions supported by Python using libraries such as `inspect` and `ast`. Summary of our schema generation process:

In [ ]:
#| echo: false
import base64
from IPython.display import HTML, display
import matplotlib.pyplot as plt

def mm(graph):
    graphbytes = graph.encode("utf8")
    base64_bytes = base64.urlsafe_b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    img_url = "https://mermaid.ink/img/" + base64_string
    
    # Responsive HTML with CSS for fitting to parent container
    html = f"""
    <div style="display: flex; justify-content: center; align-items: center; width: 100%; height: 100%;">
        <img src="{img_url}" style="max-width: 100%; max-height: 100%; object-fit: contain;" />
    </div>
    """
    display(HTML(html))

mm("""
flowchart TD
    F[Function] -->|ast| PD[Paramter descriptions]
    F -->|inspect| PT[Parameter types]
    PT --> CPT[GPT-compatible parameter types]
    F -->|basic properties| Metadata
    Metadata --> S[Schema]
    CPT --> S
    PD --> S
    E@{ shape: braces, label: "Fixup function \nService name \nExtra metadata" }
    E --> S
""")

Let us start with a well-documented function `get_weather_information`: 

In [ ]:
# Define the function to get weather information
from typing import Optional

def get_weather_information(
    city: str,  # Name of the city
    zip_code: Optional[str] = None,  # Zip code of the city (optional)
):
    """Get weather information for a city or location based on zip code"""
    return {
        "city": city,
        "zip_code": zip_code,
        "temparature": 25,
        "humidity": 80,
    }

In [ ]:
show_doc(get_weather_information)

---

### get_weather_information

>      get_weather_information (city:str, zip_code:Optional[str]=None)

*Get weather information for a city or location based on zip code*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| city | str |  | Name of the city |
| zip_code | Optional | None | Zip code of the city (optional) |

The function is well-documented and its annotations contain almost all necessary information for tool schema. We will build utilities around extracting such information and converting them into appropriate formats for tool schema.

In [ ]:
#| export
import ast
import inspect

from types import NoneType
from typing import Optional, Union, Callable, Literal,  Tuple

## Parameter descriptions

We can extract the descriptions of function parameters with the `ast` library. In our implementation, we can follow the inline comments for conveniency. The descriptions can be extracted as follows:

In [ ]:
#| export
# Extract parameter comments from the function
def extract_parameter_comments(
        func: Callable  # Function to extract comments from
    ) -> dict[str, str]:  # Dictionary with parameter comments
    """Extract comments for function arguments"""
    # Get the source code of the function
    source = inspect.getsource(func)
    # Parse the source code into an AST
    tree = ast.parse(source)
    
    # Extract comments for function arguments
    comments = {}
    for node in ast.walk(tree):
        if isinstance(node, ast.FunctionDef) and node.name == func.__name__:
            # Get arguments and comments in the function
            for arg in node.args.args:
                arg_name = arg.arg
                # Check if there's an inline comment associated with the argument
                if arg.end_lineno and arg.col_offset:
                    # Loop through the source code lines to find the comment
                    lines = source.splitlines()
                    for line in lines:
                        if line.strip().startswith(f"{arg_name}:") and "#" in line:
                            comment = line.split("#")[1].strip()
                            comments[arg_name] = comment
    return comments

In [ ]:
show_doc(extract_parameter_comments)

---

[source](https://github.com/ninjalabo/llmcam/blob/main/llmcam/core/fn_to_schema.py#L17){target="_blank" style="float:right; font-size:smaller"}

### extract_parameter_comments

>      extract_parameter_comments (func:Callable)

*Extract comments for function arguments*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| func | Callable | Function to extract comments from |
| **Returns** | **dict** | **Dictionary with parameter comments** |

Example of extracting comments from the function:

In [ ]:
extract_parameter_comments(get_weather_information)

{'city': 'Name of the city', 'zip_code': 'Zip code of the city (optional)'}

In [ ]:
#| hide
params = extract_parameter_comments(get_weather_information)
assert params == {'city': 'Name of the city', 'zip_code': 'Zip code of the city (optional)'}

## Type converter

Python types cannot be directly transferred into acceptable data types in GPT-compatible tool schema. Therefore, we need an utility to convert these types:

In [ ]:
#| export
def param_converter(
        param_type,  # The type of the parameter
        description  # The description of the parameter
    ) -> dict:  # The converted parameter
    """Convert Python parameter types to acceptable types for tool schema"""
    # Use direct mapping for simple Python types to OpenAPI types
    simple_types = {
        str: "string",
        int: "number",
        float: "number",
        bool: "boolean",
    }
    if param_type in simple_types:
        return { "type": simple_types[param_type], "description": description }
    
    # For NoneType, set the type to null and provide a suitable description
    elif param_type == NoneType:
        return { "type": "null", "description": "A default value will be automatically used." }
    
    # For list types, set the type to array and provide the item type
    if param_type == list or getattr(param_type, "__origin__", None) == list:
        item_type = param_type.__args__[0] if hasattr(param_type, "__args__") and param_type.__args__ else str
        return {
            "type": "array",
            "description": description,
            "items": { "type": param_converter(item_type, description)["type"] }
        }
    
    # For union types, set the type to anyOf and provide the subtypes
    # Note: Optional[X] is represented as Union[X, None] in Python
    if hasattr(param_type, '__origin__') and param_type.__origin__ == Union:
        # Recursively convert the types
        descriptions = description.split(" or ")
        subtypes = param_type.__args__
        if len(subtypes) > len(descriptions):
            descriptions = descriptions + ["A description is not provided"] * (len(subtypes) - len(descriptions))

        return {
            "anyOf": [param_converter(subtype, desc) for subtype, desc in zip(subtypes, descriptions)]
        }
    
    # For any other types, set the type to string and pass the description
    return { "type": "string", "description": description }

Test with parameters of `get_weather_information`:

In [ ]:
city_param = param_converter(str, "Name of the city")
zip_param = param_converter(Optional[str], "Zip code of the city (optional)")
city_param, zip_param

({'type': 'string', 'description': 'Name of the city'},
 {'anyOf': [{'type': 'string',
    'description': 'Zip code of the city (optional)'},
   {'type': 'null',
    'description': 'A default value will be automatically used.'}]})

In [ ]:
#| hide
assert city_param == {'type': 'string', 'description': 'Name of the city'}
assert zip_param == {'anyOf': [{'type': 'string', 'description': 'Zip code of the city (optional)'}, {'type': 'null', 'description': 'A default value will be automatically used.'}]}

## Function to Schema

We can combine the above utilities with other utilities in `inspect` to extract information from a Python function and generate a tool schema.

In [ ]:
#| export
def function_schema(
        func: Callable,  # The function to generate the schema for
        service_name: Optional[str] = None,  # The name of the service
        fixup: Optional[Callable] = None,  # A function to fix up the schema
        **kwargs  # Additional keyword arguments as metadata
    ) -> dict:  # The generated tool schema
    """Generate a schema from function using its parameters and docstring"""
    # Extract function name, docstring, and parameters
    func_name = func.__name__
    func_description = func.__doc__ or "No description provided."
    func_module = func.__module__
    signature = inspect.signature(func)
    
    # Create parameters schema
    parameters_schema = {
        "type": "object",
        "properties": {},
        "required": []
    }
    
    # Populate properties and required fields
    param_comments = extract_parameter_comments(func)
    for param_name, param in signature.parameters.items():
        param_type = param.annotation if param.annotation != inspect._empty else str
        
        # Add parameter to schema
        parameters_schema["properties"][param_name] = param_converter(
            param_type, 
            param_comments.get(param_name, "No description provided.")
        )
        
        # Mark as required if no default
        if param.default == inspect.Parameter.empty:
            parameters_schema["required"].append(param_name)
    
    # Build final tool schema
    tool_schema = {
        "type": "function",
        "function": {
            "name": func_name,
            "description": func_description,
            "parameters": parameters_schema,
            "metadata": {
                "module": func_module,
                "service": service_name or func_module,
                **kwargs
            }
        }
    }

    # Apply fixup function if provided
    if fixup: tool_schema['function']['fixup'] = f"{fixup.__module__}.{fixup.__name__}"
    
    return tool_schema

Test with our current function:

In [ ]:
tool_schema = function_schema(get_weather_information, service_name="Weather Service")
tool_schema

{'type': 'function',
 'function': {'name': 'get_weather_information',
  'description': 'Get weather information for a city or location based on zip code',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'Name of the city'},
    'zip_code': {'anyOf': [{'type': 'string',
       'description': 'Zip code of the city (optional)'},
      {'type': 'null',
       'description': 'A default value will be automatically used.'}]}},
   'required': ['city']},
  'metadata': {'module': '__main__', 'service': 'Weather Service'}}}

In [ ]:
#| hide
assert tool_schema == {
    'type': 'function',
    'function': {
        'name': 'get_weather_information',
        'description': 'Get weather information for a city or location based on zip code',
        'parameters': {
            'type': 'object',
            'properties': {
                'city': {'type': 'string', 'description': 'Name of the city'},
                'zip_code': {
                    'anyOf': [
                        {'type': 'string', 'description': 'Zip code of the city (optional)'},
                        {'type': 'null', 'description': 'A default value will be automatically used.'}
                    ]
                }
            },
            'required': ['city']
        },
        'metadata': {
            'module': '__main__',
            'service': 'Weather Service'
        }
    }
}

## Simulated GPT workflow

Test integrating with our current GPT framework:

In [ ]:
#| eval: false
from llmcam.core.fc import *

tools = [function_schema(get_weather_information, service_name="Weather Service")]
messages = form_msgs([
    ("system", "You can get weather information for a given location using the `get_weather_information` function"),
    ("user", "What is the weather in New York?")
])
complete(messages, tools=tools)
print_msgs(messages)

>> System:
You can get weather information for a given location using the `get_weather_information` function
>> User:
What is the weather in New York?
>> Assistant:
The current weather in New York is 25°C with 80% humidity.


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()